In [1]:
from Caida.caida_helper import *
from Ripple.ripple_helper import *
import numpy as np

Now that the data from caida and ripple are processed, it is time to find any correlation between them. <br>
The goal of this notebook is to compare nodes from both ressources. In concrete terms, I try to maximize the common IP addresses of Caida and Ripple network topology. The maximization is done by looking at different source file from Caida

In [2]:
node_file = 'Caida/midar-iff.nodes.txt'

In [3]:
caida_nodes = get_nodes(node_file)
ripple_nodes, ripple_links = get_topology()

In [4]:
# Extract IP addresses from Caida
ip_list = caida_nodes['ip'].tolist()
# Flatten the list
ip_list = [y for x in ip_list for y in x]
# Convert list in set to improve performance while searching
ip_list = set(ip_list)

In [5]:
len(ip_list), len(ripple_nodes['ip'].dropna())

(76908102, 1046)

In [9]:
# Find matches
found = 0
for ip_ripple in ripple_nodes['ip'].dropna():
    if ip_ripple in ip_list:
        found += 1

print('Percentage of matches: ' + str(found/len(ripple_nodes)))

Percentage of matches: 0.015539305301645339


To avoid code duplication, results are written here: <br>
Kapar-midar-iff -> 17/1046 <br>
Midar-iff -> 17/1046